In [ ]:
# # Install packages if needed
# install.packages("PeerPerformance", type = "source")
# install.packages("readr")
# install.packages("xtable")
# Load
library(readr)
library(PeerPerformance)
library(xtable)

set.seed(42)

# Function computing ΔSharpe triplet for one model
compare_one_model <- function(model, data, ctrl){

  base <- model
  out  <- paste0(model, " Outsider")
  ins  <- paste0(model, " Insider")

  # Tests
  t1 <- sharpeTesting(data[[out]], data[[base]], control=ctrl)     # Out−Base
  t2 <- sharpeTesting(data[[ins]], data[[base]], control=ctrl)     # In−Base
  t3 <- sharpeTesting(data[[ins]], data[[out]],  control=ctrl)     # In−Out

  d1 <- t1$dsharpe[1]; p1 <- t1$pval[1]
  d2 <- t2$dsharpe[1]; p2 <- t2$pval[1]
  d3 <- t3$dsharpe[1]; p3 <- t3$pval[1]

  # significance stars only — no p-values shown
  star <- function(p) ifelse(p<0.01,"***",
                        ifelse(p<0.05,"**",
                        ifelse(p<0.10,"*","")))

  return(c(
    sprintf("%.4f%s", d1, star(p1)),
    sprintf("%.4f%s", d2, star(p2)),
    sprintf("%.4f%s", d3, star(p3))
  ))
}


# Function generating formatted 4×3 table
run_table <- function(file){

  df <- read_csv(file, show_col_types = FALSE)

  # First column is date
  df[[1]] <- as.Date(df[[1]])
  df <- df[, -1]

  # Keep only HL series
  df <- df[, !(colnames(df) %in% c("sp500_rf"))]

  models <- c("OLS3","PLS","NN3","NN4")

  out <- matrix("", nrow=4, ncol=3)
  rownames(out) <- models
  colnames(out) <- c(
    "Δ Sharpe (Out−Base)",
    "Δ Sharpe (In−Base)",
    "Δ Sharpe (In−Out)"
  )

  ctrl <- list(
    type  = 2,
    nBoot = 499,
    bBoot = 0 # Automatic block length selection - if it doesn't work, or you get impatient, set to 4 (manual) - T^(1/3) (common heuristic) 
  )

  for(m in models){
    out[m, ] <- compare_one_model(m, df, ctrl)
  }

  return(out)
}


### EXECUTION ###
VW_table <- run_table("/Users/jonas/Documents/GitHub/Masters_Thesis_2025E/Project/Jonas/Models/H_value.csv")
EW_table <- run_table("/Users/jonas/Documents/GitHub/Masters_Thesis_2025E/Project/Jonas/Models/H_equal.csv")

# Save to CSV
write.csv(VW_table,
          file = "VW_DeltaSharpe_size_liq.csv",
          row.names = TRUE)

write.csv(EW_table,
          file = "EW_DeltaSharpe_size_liq.csv",
          row.names = TRUE)

In [ ]:
print("Value")
print(VW_table)

print("Equal")
print(EW_table)